In [ ]:
%tensorboard

In [2]:
import build_dataset

census_data = build_dataset.load_icpag_dataset(variable="ln_pred_inc_mean")

In [5]:
census_data.to_parquet("census_data.parquet")

In [17]:
import tensorflow as tf
import tensorboard as tb
import numpy as np

# Extract data from TB
experiment_id = "RAqyLXG2RMKkAXMmNEDV7Q"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
df = experiment.get_scalars()

for kind in ["train", "validation"]:
    # Keep only train values
    base = df[df.run.str.contains(f'{kind}') & (df.tag == 'epoch_loss')]
    base['run'] = base.run.str.replace(f"/{kind}", "")

    # Filter model
    models_by_date = { 
        'mobnet_v3_size128_tiles1_sample1':
            [
                'mobnet_v3_20230908-203439',
                'mobnet_v3_20230909-020833',
                'mobnet_v3_20230909-121048',
                'mobnet_v3_20230909-153231',
                'mobnet_v3_20230909-202517',
                'mobnet_v3_20230909-213040',
                'mobnet_v3_20230910-130103',
                'mobnet_v3_20230911-092415',
                'mobnet_v3_20230911-231611',
                'mobnet_v3_20230912-012855',
            ],
        'mobnet_v3_size256_tiles1_sample1':
            [       
                'mobnet_v3_20230914-215006',
                'mobnet_v3_20230913-200010',
            ],      
        'mobnet_v3_size512_tiles1_sample1':
            [
                'mobnet_v3_20230915-122541',
                'mobnet_v3_20230916-124818',
            ],
        'mobnet_v3_size128_tiles2_sample20':
            [
                'mobnet_v3_20230918-192554',
                'mobnet_v3_20230918-231134',
                'mobnet_v3_20230919-104912',
                'mobnet_v3_20230919-234505',
                'mobnet_v3_20230920-131123',
                'mobnet_v3_20230920-180458',
                'mobnet_v3_20230920-194725',
            ]
    }

    base['model'] = np.nan
    for model, names in models_by_date.items():
        for name in names:
            base.loc[base.run.str.contains(name), 'model'] = model
            
    # Reshape data
    plot_data = base[~base.model.isna()]\
        .drop_duplicates(subset=['model','step'], keep='first')\
        .pivot(index='step', columns='model', values='value')
        
    # Export data
    path_repo = r"D:/Maestría/Tesis/Repo"
    plot_data.to_csv(f"{path_repo}/data/data_out/{kind}_by_epoch.csv")

C:\Users\ofici\AppData\Local\Temp\ipykernel_12596\1209519343.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base['run'] = base.run.str.replace(f"/{kind}", "")
C:\Users\ofici\AppData\Local\Temp\ipykernel_12596\1209519343.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base['model'] = np.nan
C:\Users\ofici\AppData\Local\Temp\ipykernel_12596\1209519343.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [22]:
kind

'validation'

In [20]:
plot_data

model,mobnet_v3_size128_tiles1_sample1,mobnet_v3_size128_tiles2_sample20,mobnet_v3_size256_tiles1_sample1,mobnet_v3_size512_tiles1_sample1
step,,,,
0,NaN,0.816814,0.818951,0.810019
1,NaN,0.814255,0.824106,0.827106
2,NaN,0.803770,0.809365,0.806643
3,NaN,70.086678,0.940823,0.880816
4,NaN,7670.543457,0.845501,1.634859
...,...,...,...,...
195,0.491770,0.419588,0.530523,0.329793
196,0.472416,0.476143,0.332168,0.286581
197,0.498945,0.495958,0.600452,0.429535


In [21]:
base

,run,tag,step,value,model
64,mobnet_v3_size128_tiles1_sample1/mobnet_v3_202...,epoch_loss,14,0.837110,mobnet_v3_size128_tiles1_sample1
65,mobnet_v3_size128_tiles1_sample1/mobnet_v3_202...,epoch_loss,15,1.043880,mobnet_v3_size128_tiles1_sample1
66,mobnet_v3_size128_tiles1_sample1/mobnet_v3_202...,epoch_loss,16,0.758083,mobnet_v3_size128_tiles1_sample1
67,mobnet_v3_size128_tiles1_sample1/mobnet_v3_202...,epoch_loss,17,0.628917,mobnet_v3_size128_tiles1_sample1
68,mobnet_v3_size128_tiles1_sample1/mobnet_v3_202...,epoch_loss,18,0.634569,mobnet_v3_size128_tiles1_sample1
...,...,...,...,...,...
7295,mobnet_v3_size512_tiles1_sample1/mobnet_v3_202...,epoch_loss,195,0.329793,mobnet_v3_size512_tiles1_sample1
7296,mobnet_v3_size512_tiles1_sample1/mobnet_v3_202...,epoch_loss,196,0.286581,mobnet_v3_size512_tiles1_sample1
7297,mobnet_v3_size512_tiles1_sample1/mobnet_v3_202...,epoch_loss,197,0.429535,mobnet_v3_size512_tiles1_sample1
7298,mobnet_v3_size512_tiles1_sample1/mobnet_v3_202...,epoch_loss,198,0.395025,mobnet_v3_size512_tiles1_sample1


In [1]:
import importlib
import pandas as pd
import tensorflow as tf

import utils
import build_dataset as bd
import run_model

importlib.reload(run_model)
importlib.reload(utils)

path_repo = r"/mnt/d/Maestría/Tesis/Repo/"

log_dir = f"{path_repo}/logs/mobnet_v3_20230905-231447"
models_dir = f"{path_repo}/data/data_out/models_by_epoch"
model_name = "mobnet_v3"
metadata = pd.read_csv(f"{path_repo}/data/data_out/size128_sample10/metadata.csv")
tiles = 1
size = 128
resizing_size = 128
bias = 2
sample = 1
to8bit = True
n_epochs = 20


# modelpath = rf"{path_repo}/data/data_out/models_by_epoch/mobnet_v3_25"
# model = tf.keras.models.load_model(
#            modelpath, compile=True
#         )
# metadata = metadata.loc[metadata.type == "test", "link"]
# df_prediciones, mse = run_model.compute_custom_loss(
#     model, metadata, tiles, size, resizing_size, bias, sample, to8bit
# )

KeyboardInterrupt: 

In [ ]:
%tensorboard

In [3]:
import xarray as xr
import os
path_pansharpened = r"D:\Maestría\Tesis\Repo\data\data_in\Pansharpened\2013"
files = os.listdir(path_pansharpened)
tifs = [f for f in files if f.endswith(".tif")]
tifs

ds1 = xr.open_dataset(rf"{path_pansharpened}\{tifs[0]}")
ds2 = xr.open_dataset(rf"{path_pansharpened}\{tifs[1]}")

In [7]:
xr.combine_by_coords([ds1, ds2])

In [18]:
df_prediciones, mse = run_model.compute_custom_loss(
    model, metadata[metadata.link == int(64100610)], tiles, size, resizing_size, bias, sample, to8bit
)

064100610: 0/1
1/1 [==============================] - 1s 1s/step


In [ ]:
import importlib
import build_dataset
import run_model
import utils
import matplotlib
import geopandas as gpd
from shapely import Polygon
importlib.reload(build_dataset)
importlib.reload(utils)

link = '064100610'

# Load datasets
datasets, extents = build_dataset.load_satellite_datasets()
icpag = build_dataset.load_icpag_dataset()
icpag = build_dataset.assign_links_to_datasets(icpag, extents, verbose=False)

# Get the dataset with the images of the selected link
ds = build_dataset.get_dataset_for_link(icpag, datasets, link)

# Get the grid of the images
images, points, bounds = build_dataset.get_gridded_images_for_link(
    ds, icpag, link, tiles=1, size=128, resizing_size=128, bias=4, sample=1, to8bit=True
)

# Compute loss
df_prediciones, mse = run_model.compute_custom_loss(
    model, metadata[metadata.link == int(link)], tiles, size, resizing_size, bias, sample, to8bit
)

# Make geodataframe with the images and its predictions
polygons = [Polygon(bound[0]) for bound in bounds]
predictions = df_prediciones[df_prediciones['link']==link].predictions.values[0]
predictions_gdf =  gpd.GeoDataFrame(predictions, geometry=polygons).rename(columns={0:'predictions'}).set_crs(epsg=4326)

#### Plot
import folium
# Plot census tract
m = icpag[icpag.link == link].explore(
        tiles="https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr="ESRI",
        
)
# Plot gridded predictions
predictions_gdf.explore(column='predictions', cmap='Spectral', vmin=-1, vmax=1, m=m)

# Add control for switching between layers
folium.LayerControl().add_to(m)
m

In [10]:
import tensorflow as tf
import run_model
import pandas as pd
import importlib
importlib.reload(run_model)

path_repo = r"/mnt/d/Maestría/Tesis/Repo/"
modelpath = rf"{path_repo}/data/data_out/models_by_epoch/mobnet_v3_25"
# model = tf.keras.models.load_model(
#            modelpath, compile=True
#         )
# metadata = pd.read_csv(f"{path_repo}/data/data_out/train_size128_tiles1_sample10/metadata.csv")
tiles, size, resizing_size, bias, sample = 1, 128, 128, 2, 1

df, mse = run_model.compute_custom_loss(
    model, metadata, tiles, size, resizing_size, bias, sample, True, verbose=True
)a
mse

0.42271230232120466

In [13]:
metadata.loc[metadata.link ==10 ].icpag

NameError: name 'metadata' is not defined

In [ ]:
predictions_gdf.explore(column='predictions', cmap='Spectral', vmin=-1, vmax=1)

In [9]:
import numpy as np
np.concatenate([np.array([10,20,30]),np.array([10])]).shape

(4,)

In [ ]:
bounds[0][0]

In [ ]:
bounds[0]

In [ ]:
colorize(prediction, vmin=vmin, vmax=vmax)

In [ ]:
import pandas as pd
metadata = pd.read_csv(r"D:\Maestría\Tesis\Repo\data\data_out\size512_sample5\metadata.csv")

In [ ]:
link = 20010101
metadata.loc[metadata.link == link, 'var'].values[0]

In [ ]:
import os
os.getcwd()

In [ ]:
import pandas as pd
pd.read_csv("./data/icpag.csv").head()

In [ ]:
from tensorflow import keras
path_dataout = r"D:\Maestría\Tesis\Repo\data\data_out"
model_path=fr"{path_dataout}/models/mobnet_v3_20230831-172738"

keras.models.load_model(model_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
img = np.load(r"D:\Maestría\Tesis\Repo\data\data_out\size128_sample10\020010103_1.npy")
img = np.moveaxis(img, 0, 2)[:,:,:3]
# equalize hist
img = skimage.exposure.equalize_hist(img)
plt.imshow(img)

In [ ]:
len(images)

In [ ]:
import earthpy.plot as ep

# ep.plot_rgb(images[0], rgb=[0, 1, 2], title="RGB Image", stretch=True)


In [ ]:
composition

In [ ]:
import pandas as pd
from dotenv import dotenv_values
image_size = 200
sample_size = 1

pd.set_option("display.max_columns", None)
env = dotenv_values("D:/Maestría/Tesis/Repo/scripts/globals.env")

path_proyecto = env["PATH_PROYECTO"]
path_datain = env["PATH_DATAIN"]
path_dataout = env["PATH_DATAOUT"]
path_scripts = env["PATH_SCRIPTS"]
path_satelites = env["PATH_SATELITES"]
path_logs = env["PATH_LOGS"]
path_outputs = env["PATH_OUTPUTS"]

# df = pd.read_csv(
#     rf"D:\Maestría\Tesis\Repo\data\data_out\size500_sample1\metadata.csv"
# )


In [ ]:
import importlib
import build_dataset
importlib.reload(build_dataset)
variable = "ln_pred_inc_mean"
datasets, extents = build_dataset.load_satellite_datasets()
icpag = build_dataset.load_icpag_dataset(variable)
icpag = build_dataset.assign_links_to_datasets(icpag, extents)



In [ ]:
import run_model
import importlib
importlib.reload(run_model)

kind="reg"
image_size = 500
resizing_size = 200
my_test = "ddddddd"
train_dataset, test_dataset, filenames = run_model.create_and_build_datasets(
    kind=kind,
    image_size=image_size,
    resizing_size=resizing_size,
    batch_size=32,
)



In [ ]:
test["type"].value_counts()

In [ ]:
icpag['type'].value_counts()

In [ ]:
import numpy as np
imgs = np.load(r"D:\Maestría\Tesis\Repo\outputs\examples_0_img.npy")
labs = np.load(r"D:\Maestría\Tesis\Repo\outputs\examples_0_lab.npy")

In [ ]:
def process_image(file_path, label):
    import cv2
    
    img = np.load(file_path)
    img = np.moveaxis(
        img, 0, 2
    )  # Move axis so the original [4, 512, 512] becames [512, 512, 4]
    img = cv2.resize(
        img, dsize=(resizing_size, resizing_size), interpolation=cv2.INTER_CUBIC
    )

    img = tf.convert_to_tensor(img / 255, dtype=tf.float32)
    label = tf.cast(label, tf.float32)

    return img, label

In [ ]:
import tensorflow as tf
import numpy as np
resizing_size = 200
img, label = process_image(r"D:\Maestría\Tesis\Repo\data\data_out\size500_sample1\062742004_0.npy", 10)

In [ ]:
import tensorflow_datasets as tfds
example = tfds.as_numpy(img)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(example[:,:,0:3])

In [ ]:
import os
path = r"D:\Maestría\Tesis\Repo\data\data_out\size500_sample1"
files = [f"{path}\{file}" for file in os.listdir(path)]

labels = [0] * len(files)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((files, labels))


In [ ]:
dataset = dataset.map(
    lambda file, label: tf.numpy_function(
        process_image, [file, label], (tf.float32, tf.float32)
    )
)  # Parse every image in the dataset using `map`

imgs = []
labs = []
for x in dataset.take(1):
    imgs += [tfds.as_numpy(x)[0]]
    labs += [tfds.as_numpy(x)[1]]
    
imgs
#### HASTA ACA TODO GENIAL


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
        [
            layers.RandomFlip(
                "horizontal_and_vertical",
                seed=825,
                input_shape=(resizing_size, resizing_size, 4),
            ),
            # layers.RandomTranslation(0.3, 0.3, fill_mode="reflect", seed=825),
            # layers.RandomHeight(0.3),
            # layers.RandomWidth(0.3),
            # layers.RandomZoom(0.3, seed=825),
            # layers.RandomContrast(0.3, seed=825),
            # layers.RandomBrightness(0.05, value_range=(0,1), seed=825),
            # layers.RandomCrop(image_size, image_size, seed=825),
        ],
        name="data_augmentation",
    )

In [ ]:
train_dataset = (
    dataset.shuffle(round(len(files[0]) / 10))
    .batch(32)
    .map(lambda x, y: (data_augmentation(x), y))
    .prefetch(tf.data.AUTOTUNE)
)


In [ ]:
import utils

ds = datasets[icpag.loc[icpag.link == "065150214", "dataset"].values[0]]
composition, point, boundaries, total_boundaries = utils.random_image_from_census_tract(
    ds, icpag, "065150214", tiles=2, size=500, bias=4, to8bit=True
)

In [ ]:
total_boundaries

In [ ]:
imgs = []
labs = []
for x in train_dataset.take(1):
    imgs += [tfds.as_numpy(x)[0]]
    labs += [tfds.as_numpy(x)[1]]

print(imgs[0][0].shape)    
plt.imshow(imgs[0][0][:,:,:3])

In [ ]:
sample_size = 1
image_size = 500
path_dataout = r"D:/Maestría/Tesis/Repo/data/data_out"

df = pd.read_csv(
    rf"{path_dataout}/size{image_size}_sample{sample_size}/metadata.csv"
)


In [ ]:
def point_column_to_x_y(df):
    df.point = df.point.str.replace("\(|\)", "", regex=True).str.split(",")
    df['x'] = df.point.str[0]
    df['y']= df.point.str[1]
    return df[['x', 'y']]

In [ ]:
import geopandas as gpd
df = df.dropna(how="any").reset_index(drop=True)
df[['x', 'y']] = point_column_to_x_y(df)
df.x, df.y = df.x.astype(float), df.y.astype(float)


In [ ]:
df

In [ ]:
import build_dataset
import pandas as pd
path_dataout = r"D:/Maestría/Tesis/Repo/data/data_out"

# build_dataset.build_dataset(200, 1, variable="ln_pred_inc_mean")
df = pd.read_csv(
    rf"{path_dataout}/size200_sample1/metadata.csv"
)

metadata = build_dataset.split_train_test(df)

In [ ]:
import importlib
import build_dataset
import utils
importlib.reload(build_dataset)
importlib.reload(utils)
importlib.reload(build_dataset)
build_dataset.build_dataset(500, 1, tiles=2, bias=4, variable="ln_pred_inc_mean")

In [ ]:
import importlib
importlib.reload(build_dataset)
df = pd.read_csv(
    rf"{path_dataout}/size500_sample1/metadata.csv"
)
# df = df.drop(columns="Test")
# df = build_dataset.split_train_test(df)